In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
data=pd.read_csv('data.csv')

## Concatenate the Data Frames
X=data.iloc[:,3:13]
Y=data.iloc[:,13]

In [4]:
#Create dummy variables
geography=pd.get_dummies(X["Geography"],drop_first=True)
gender=pd.get_dummies(X['Gender'],drop_first=True)
X=pd.concat([X,geography,gender],axis=1)
X=X.drop(['Geography','Gender'],axis=1)


In [5]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)


In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout
from keras.layers import BatchNormalization


In [7]:
classifier=Sequential()
classifier.add(Dense(units=6,kernel_initializer='lecun_normal',activation='relu',input_dim=11))
classifier.add(BatchNormalization())
classifier.add(Dense(units=6,kernel_initializer='lecun_normal',activation='selu'))
classifier.add(BatchNormalization())
classifier.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))
classifier.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])


In [ ]:
model=classifier.fit(X_train,Y_train,validation_split=0.2,batch_size=64,epochs=100)

In [ ]:
y_pred=classifier.predict(X_test)
y_pred=(y_pred>0.5)
from sklearn.metrics import accuracy_score
score=accuracy_score(y_pred,Y_test)
score

In [ ]:
plt.plot(model.history['accuracy'])
plt.plot(model.history['val_accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'],loc='upper left')
plt.show()

In [22]:
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch

In [ ]:
# keras Tuner
def build_model(hp):
    model=keras.Sequential()
    model.add(layers.Flatten())
    for i in range(hp.Int('num_layers',2,10)):
        model.add(
        Dense(
        units=hp.Int(f'units_{i}',min_value=32,max_value=128,step=32),
        activation='relu',
        kernel_initializer='he_uniform')
        )
        model.add(BatchNormalization())

    #if hp.Boolean('dropout'):
    #    model.add(layers.Dropout(rate=0.25))
    model.add(Dense(units=1,activation='sigmoid',kernel_initializer='glorot_uniform'))
    learning_rate = 0.001
    model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
    loss='binary_crossentropy',
    metrics=['accuracy']
    )
    return model

In [25]:
tuner = RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=20,
    executions_per_trial=5,
    overwrite=True,
    directory="Project_1",
    project_name="Test",
)

In [ ]:
tuner.search(X_train, Y_train, epochs=10,validation_split=0.2,batch_size=64)

In [ ]:
tuner.results_summary()

In [ ]:
models = tuner.get_best_models(num_models=5)
best_model = models[0]
best_model.build(input_shape=(None,11))
best_model.summary()

In [ ]:
y_pred=best_model.predict(X_test)
y_pred=(y_pred>0.5)
score=accuracy_score(y_pred,Y_test)
score